# Step 2

In [151]:
import pydot
import matplotlib.pyplot as plt
import time
import math
import random
import numpy as np
from scipy.special import comb
#import networkx as nx

In [152]:
%run '/Users/tomascarrilho/Desktop/Faculdade/Data Visualization/Project/Data_Visualization/Step1.ipynb'

# Trees

## First Layout

In [153]:
def layered_ish(file): #node_positions
    '''Returns the node positions {node:(x,y) and also the final tree (edges and stuff)
    '''
    total = 0
    n=0
    graph = get_data(file)
    nodes = graph[0]
    node_positions = {node.get_name(): (0, 0) for node in nodes}
    done_nodes = set()
    node_childs = f_childs(file)[1]
    min_y = 0

    for node in nodes:
        done_nodes.add(str(node.get_name()))
        childs = node_childs[str(node.get_name())]
        parent_x = node_positions[str(node.get_name())][0]
        parent_y = node_positions[str(node.get_name())][1]
        step = 3

        if len(node_childs[str(node.get_name())]) == 1 and str(node_childs[str(node.get_name())][0]) not in done_nodes:
            child_x = parent_x + step
            child_y = parent_y
            node_positions[str(node_childs[str(node.get_name())][0])] = (child_x, child_y)
            done_nodes.add(str(node_childs[str(node.get_name())][0]))
            
        elif len(node_childs[str(node.get_name())]) >1:
            new_childs = len([child for child in node_childs[str(node.get_name())] if child not in done_nodes])
            max_y = (new_childs-1)*3
            
            done_childs = []
            for child in [new_child for new_child in node_childs[str(node.get_name())] if new_child not in done_nodes]:
                done_childs.append(child)
                child_x = parent_x + step
                child_y = parent_y + max_y
                
                node_positions[child] = (child_x, child_y)
                done_nodes.add(str(child))
                max_y -= 6

            if len(done_childs)>0:
                min_y = node_positions[done_childs[-1]][1]
    return node_positions

# write_ordered_dic('Datasets/LesMiserables.dot')
# tree = layered_ish('Datasets/LesMiserables.dot')
# tree

## BFS tree

In [154]:
def BFS_tree(file): #final_tree: dic
    '''Returns the BFS tree (edges and stuff) {parent:[childs]}
    '''
    graph = get_data(file)
    nodes = graph[0]
    done_nodes = set()
    node_childs = f_childs(file)[1]
    tree = {node.get_name(): [] for node in nodes}
    
    for node in nodes:
        done_nodes.add(str(node.get_name()))
        done_childs = []
        for child in [new_child for new_child in node_childs[str(node.get_name())] if new_child not in done_nodes]:
            done_childs.append(child)
            done_nodes.add(str(child))
            tree[node.get_name()].append(str(child))    
    final_tree = {i:j for i,j in tree.items() if j!= []}
    return final_tree

#BFS_tree('ordered_LesMiserables.dot')
#BFS_tree('Datasets/LesMiserables.dot')

## DFS tree

In [155]:
def tree_layout_DFS(file): #we need to change the 1 in the return value
    #returns the coordinates for each node 
    graph = get_data(file)
    return dfs({node.get_name(): (0, 0) for node in graph[0]}, 1, node_childs=f_childs(file)[1], visited_nodes=[], edges=graph[1], tree= {node.get_name(): [] for node in graph[0]})



def dfs(node_positions, node, node_childs, visited_nodes, edges, tree, x=0, y=0): #node_positions, final_tree
    visited_nodes.append(node)
    parent_x, parent_y = x, y
    
    step_x = 1  
    step_y = 2  

    if len(node_childs[str(node)]) == 0:
        return node_positions

    child_y = parent_y - step_y
    total_width = step_x * (len(node_childs[str(node)]) - 1)
    start_x = parent_x - total_width / 2
    
    for i, child in enumerate(node_childs[str(node)]):
        if child not in visited_nodes:
            child_x = start_x + i * step_x
            node_positions[str(child)] = (child_x, child_y)

            tree[str(node)].append(child)
            dfs(node_positions, child, node_childs, visited_nodes, edges, tree, child_x, child_y)
    
    final_tree = {i:j for i,j in tree.items() if j!= []}
    return node_positions, final_tree

#print(tree_layout_DFS('Datasets/LesMiserables.dot')[0])
#show_graph('Datasets/LesMiserables.dot','layered_ish' ,'DFS',(18,18))

#print(tree_layout_DFS('Datasets/LesMiserables.dot')[1])
# show_graph('Datasets/LesMiserables.dot','layered_ish' ,'DFS',(18,18))

#ans = tree_layout_DFS('ordered_LesMiserables.dot')[1]

In [156]:
def dfs1(graph, start_node, visited=None, dfs_tree=None):
    """
    Performs Depth-First Search (DFS) traversal starting from the given start node.

    Parameters:
        graph (dict): Dictionary representing the graph.
        start_node (str): The starting node for the DFS traversal.
        visited (set): Set to keep track of visited nodes. Default is an empty set.
        dfs_tree (dict): Dictionary representing the DFS tree. Default is an empty dictionary.

    Returns:
        dict: Dictionary representing the DFS tree.
    """
    if visited is None:
        visited = set()
    if dfs_tree is None:
        dfs_tree = {}

    visited.add(start_node)

    for neighbor in graph.get(start_node, []):
        if neighbor not in visited:
            if start_node not in dfs_tree:
                dfs_tree[start_node] = []
            dfs_tree[start_node].append(neighbor)
            dfs(graph, neighbor, visited, dfs_tree)

    return dfs_tree

def DFS_tree1(file, start_node=None):
    """
    Constructs a DFS tree from a list of nodes and edges.

    Parameters:
        nodes (list): List of node labels.
        edges (list): List of tuples representing edges (source, target).
        start_node (str, optional): The starting node for the DFS traversal. If None, the first node in the list of nodes will be used.

    Returns:
        dict: Dictionary representing the DFS tree.
    """
    nodes = get_data(file)[2]
    edges = get_data(file)[3]
    graph = {}
    for edge in edges:
        source, target = edge
        if source not in graph:
            graph[source] = []
        graph[source].append(target)

    if start_node is None:
        start_node = nodes[0]

    final_tree = dfs(graph, start_node)


    return final_tree


## Radial Layout

### Auxiliary Functions

In [157]:
def sort_nodes_degree(file): #Returns a sorted dictionary containing the nodes, as keys, and a list of it's childs, as values, ordered based on the degree (> to <)
    '''Returns a sorted dictionary containing the nodes, as keys, and a list of it's childs, as values, ordered based on the degree (> to <)
    '''
    return dict(sorted(f_edges(file)[1].items(), key=lambda item: len(item[1]), reverse=True))

# dic = sort_nodes_degree('Datasets/LesMiserables.dot')
# dic

def write_ordered_dic(file,new_file): #writes a new .dot file but with the nodes ordered based of their degree
    '''writes a new .dot file but with the nodes ordered based of their degree
    '''
    dic = sort_nodes_degree(file)
    nodes_list = get_data(file)[2]
    edge_list = get_data(file)[3]
    weigths = get_data(file)[4]
    done_nodes = set()

    graph = pydot.Dot(graph_type='graph')

    # for node_ish in nodes_list:
    #     node = pydot.Node(node_ish)
    #     graph.add_node(node)
    
    for node_ish,childs in dic.items():
        node = pydot.Node(node_ish)
        if node_ish not in done_nodes:
            graph.add_node(node)
            done_nodes.add(node_ish)
        for child_ish in childs:
            child = pydot.Node(child_ish)
            if child_ish not in done_nodes:
                graph.add_node(child)
                done_nodes.add(child_ish)
            #graph.add_node(child)
            edge_ish = (str(node_ish), str(child_ish) )
            if edge_ish not in edge_list:
                edge_ish = (str(child_ish), str(node_ish))
            w = 1
            if weigths!= {}:
                w = int(weigths[edge_ish])
            edge = pydot.Edge(str(node_ish), str(child_ish), weight=w)
            graph.add_edge(edge)
    
    graph.write(new_file)


def get_key(dictionary, value): #Return the key of a specific value. 
    '''Return the key of a specific value. 
    Changed the function while working on the node_positioning function to work with tuples. Need to check
    whether this affects other functions
    '''
    if isinstance(value, tuple):
        return next((key for key, val in dictionary.items() if val == value), None)
    else:
        return next((key for key, val in dictionary.items() if value in val), None)

def count_descendants(tree, node_name): #descendants
    'returns the number (int) of descendants of a node'
    if node_name not in tree:
        return 0
    
    descendants = 0
    if node_name in tree:
        children = tree[node_name]
        descendants += len(children)  # Add the direct children count
        for child in children:
            descendants += count_descendants(tree, child)
    return descendants

def count_ancestors(tree, node_name): #ancestors
    'returns the number (int) of ancestors of a node'
    ancestors = 0
    nodes = tree
    key = get_key(tree,node_name)
    if key != None:
        parent = get_key(tree,node_name)
        ancestors += 1           
        ancestors += count_ancestors(tree, parent)
    return ancestors

def f_layers(file,layout='BFS'): #Organizes the nodes based on the layer their own on a list of lists(index = layer)
    '''Organizes the nodes based on the layer their own on a list of lists(index = layer)
    '''
    layers = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]

    if layout == 'BFS':
        tree = BFS_tree(file)
        node_positions = layered_ish(file)
        for node, childs in tree.items():
            layers.append([])
            for child in childs:
                layer = int(node_positions[child][0]/3)
                layers[layer].append(child)
    if layout == 'DFS':
        node_positions = tree_layout_DFS(file)[0]
        tree = tree_layout_DFS(file)[1]
        for node, childs in tree.items():
            layers.append([])
            for child in childs:
                layer = -int(node_positions[child][1]/2)
                layers[layer].append(child)
    root = list(tree.keys())[0]
    layers[0].append(root)
    return [layer for layer in layers if layer != []]
#print(f_layers('ordered_LesMiserables.dot',layout='DFS'))

In [158]:
def create_pydot_graph(nodes, edges, filename,edge_weights={},dir = False): #created a .dot file
    if dir == False:
        graph = pydot.Dot(graph_type='graph')
    
    else: 
        graph = pydot.Dot(graph_type='digraph')

    # Add nodes to the graph
    for node in nodes:
        graph.add_node(pydot.Node(node))

    # Add edges to the graph
    for edge in edges:
        src, dest = edge
        weight = edge_weights.get(edge, 0) 
        graph.add_edge(pydot.Edge(src, dest, label=str(weight)))
        

    graph.write(filename)

### Radial Implementation

In [159]:
def radial(file,tree_type = 'BFS',ordesection = False): #node_positions
    '''returns the nodes\' positions if we apply the radial layout implementation
    '''
    if ordesection != False:
        new_file = write_ordesection_dic(file)
        file = new_file

    data = get_data(file)
    nodes,nodes_list,edge_list,weigths = data[0],data[2],data[3],data[4]
    l_radius = []

    if tree_type == 'DFS':
        tree = tree_layout_DFS(file)[1]
        layers = f_layers(file,layout='DFS')
    
    elif tree_type == 'BFS':
        tree = BFS_tree(file)
        layers = f_layers(file,layout='BFS')

    node_positions = {node.get_name(): (0, 0) for node in nodes}
    root = list(tree.keys())[0]

    start_angle=0
    for subtree in tree:
        angle = 0
        for node in tree[subtree]:
            for i in layers:
                if node in i:
                    layer = layers.index(i)

            radius = 3*layer
            l_radius.append(radius)
            parent_x = node_positions[get_key(tree,node)][0]
            parent_y = node_positions[get_key(tree,node)][1]

            #angle for the section for each node
            section = 2*math.pi*(1+count_descendants(tree, subtree))/count_descendants(tree, root)
            #increment between sibling nodes
            increment = 2*math.pi*(count_descendants(tree, node)+1)/count_descendants(tree, root)
            
            if layer > 1 and tree[subtree].index(node)==0:
                #the starting angle must be the parent angle - half of section              
                start_angle = math.acos(parent_x/(3*(layer-1)))-section/2
                if parent_y <0:
                    #acos has domain [0,pi], so we need to fix it where x>pi
                    start_angle = -math.acos(parent_x/(3*(layer-1)))-section/2
                angle+= start_angle
            
            angle+= increment/2
            child_x = math.cos(angle)*radius
            child_y = math.sin(angle)*radius
            angle+= increment/2
            node_positions[node]=(child_x,child_y)
   
    return node_positions,l_radius

#radial('Datasets/LesMiserables.dot',tree_type='DFS')

# Plotting

In [175]:
#HOW TO PLOT A GENERAL GRAPH
def show_graph(file,layout,tree='BFS', dims=(18,18),start_node=1):
    edge_list,weigths = get_data(file)[3],get_data(file)[4]

    if layout == 'tree_layout':
        node_positions = tree_layout(file)[0]
        edges = get_data(file)[1]
    if layout == 'tree_layout_improv':
        node_positions = tree_layout_improv(file)
        edges = get_data(file)[1]

    if layout == 'layered_ish':
        node_positions = layered_ish(file)
    
    
    if tree == 'BFS' and layout in ['layered_ish','radial']:
        edges_ish = BFS_tree(file)
        edges = []
        for source,l_dest in edges_ish.items():
            for dest in l_dest:
                edges.append((source,dest))
        if layout == 'radial':
            node_positions, l_radius = radial(file,tree_type='BFS')

    if tree == 'DFS' and layout in ['layered_ish','radial']:
        tree_org = tree_layout_DFS(file)
        edges_ish = tree_org[1]
        node_positions = tree_org[0]
        edges = []
        for source,l_dest in edges_ish.items():
            for dest in l_dest:
                edges.append((source,dest))
        if layout == 'radial':
            node_positions,l_radius = radial(file,tree_type='DFS')

    plt.figure(figsize=(dims[0], dims[1]))
    w = 1
    for edge in edges:
        if layout not in ['layered_ish','radial']:
            start_node = edge.get_source()
            end_node = edge.get_destination()
            position = str(str(edge.get_attributes())).rfind(':')
            if int(position) != -1:
                w = int(str(edge.get_attributes())[position+3:-2])
                if w != 1:
                    w = w/10
        else:
            start_node = edge[0]
            end_node = edge[1]
            new_edge = (start_node,end_node)
            if new_edge not in edge_list:
                new_edge = (end_node,start_node)
            if weigths !={}:
                w = weigths[new_edge]
                if w != 1:
                    w=w/10

        start_pos = node_positions[start_node]
        end_pos = node_positions[end_node]
        plt.plot([start_pos[0], end_pos[0]], [start_pos[1], end_pos[1]], 'k-',linewidth=w,zorder = 2) 

    for node, pos in node_positions.items():
        plt.scatter(pos[0], pos[1], color='green', s=200, marker='o', zorder =3)  
        plt.text(pos[0], pos[1], str(node), fontsize=10, ha='center', va='center')
    
    if layout == 'radial':
        for radius in l_radius:
            plt.plot(radius * np.cos(np.linspace(0, 2*np.pi, 100)), radius * np.sin(np.linspace(0, 2*np.pi, 100)), color='gray', alpha=0.2,zorder = 1)
        

    plt.axis('off')
    plt.gca().set_facecolor('w')
    plt.savefig('Plots/'+tree+'_'+layout+'_plot.png')
    plt.show()
    
#show_graph('Datasets/LesMiserables.dot','tree_layout1',(18,18))
#show_graph('Datasets/LesMiserables.dot','radial',tree = 'DFS')

In [ ]:
#PLOT GRAPH WITH INVALID EDGES
def show_graph_with_invalid_edges(file,layout,tree='BFS', dims=(18,18),start_node=1):
    edge_list,weigths = get_data(file)[3],get_data(file)[4]
    all_edges = get_data(file)[1]
    valid_nodes = []
    if layout == 'tree_layout':
        node_positions = tree_layout(file)
        edges = get_data(file)[1]
    if layout == 'tree_layout_improv':
        node_positions = tree_layout_improv(file)
        edges = get_data(file)[1]

    if layout == 'layered_ish':
        node_positions = layered_ish(file)
    
    
    if tree == 'BFS' and layout in ['layered_ish','radial']:
        edges_ish = BFS_tree(file)
        edges = []
        for source,l_dest in edges_ish.items():
            for dest in l_dest:
                edges.append((source,dest))
        if layout == 'radial':
            node_positions, l_radius = radial(file,tree_type='BFS')

    if tree == 'DFS' and layout in ['layered_ish','radial']:
        tree_org = tree_layout_DFS(file)
        edges_ish = tree_org[1]
        node_positions = tree_org[0]
        edges = []
        for source,l_dest in edges_ish.items():
            for dest in l_dest:
                edges.append((source,dest))
        if layout == 'radial':
            node_positions,l_radius = radial(file,tree_type='DFS')

    plt.figure(figsize=(dims[0], dims[1]))
    w = 1
    for edge in edges:
        if layout not in ['layered_ish','radial']:
            start_node = edge.get_source()
            end_node = edge.get_destination()
            position = str(str(edge.get_attributes())).rfind(':')
            if int(position) != -1:
                w = int(str(edge.get_attributes())[position+3:-2])
                if w != 1:
                    w = w/10
        else:
            start_node = edge[0]
            end_node = edge[1]
            new_edge = (start_node,end_node)
            if new_edge not in edge_list:
                new_edge = (end_node,start_node)
            if weigths !={}:
                w = weigths[new_edge]
                if w != 1:
                    w=w/10

        start_pos = node_positions[start_node]
        end_pos = node_positions[end_node]
        valid_nodes.append((start_node, end_node))
        plt.plot([start_pos[0], end_pos[0]], [start_pos[1], end_pos[1]], 'k-',linewidth=w,zorder = 2) 

    for node, pos in node_positions.items():
        plt.scatter(pos[0], pos[1], color='green', s=200, marker='o', zorder =3)  
        plt.text(pos[0], pos[1], str(node), fontsize=10, ha='center', va='center')
    
    if layout == 'radial':
        for radius in l_radius:
            plt.plot(radius * np.cos(np.linspace(0, 2*np.pi, 100)), radius * np.sin(np.linspace(0, 2*np.pi, 100)), color='gray', alpha=0.2,zorder = 1)
        
    
    #add the invalid nodes
    for e in all_edges:
        if ((str(e.get_source()), str(e.get_destination())) or (str(e.get_destination()), str(e.get_source()))) not in valid_nodes:
            print(f"Invalid edge: {e.get_source()}, {e.get_destination()}")
            start_node = e.get_source()
            end_node = e.get_destination()

            start_pos = node_positions[start_node]
            end_pos = node_positions[end_node]
            plt.plot([start_pos[0], end_pos[0]], [start_pos[1], end_pos[1]], 'k-',linewidth=0.5,zorder = 1, color="red")

    plt.axis('off')
    plt.gca().set_facecolor('w')
    plt.savefig('Plots/'+tree+'_'+layout+'_plot.png')
    plt.show()

In [ ]:
show_graph_with_invalid_edges("Datasets/LesMiserables.dot", tree="DFS",  layout="radial")
show_graph_with_invalid_edges("Datasets/LesMiserables.dot", tree="BFS",  layout="radial")

# Results

## Initial Layout

In [177]:
#show_graph('Datasets/LesMiserables.dot','tree_layout')

## Improved Initial Layout

In [178]:
#show_graph('Datasets/LesMiserables.dot','tree_layout_improv')

## BFS

In [162]:
#show_graph('Datasets/LesMiserables.dot','layered_ish')

By ordering the nodes based on their degree to select the best root node

In [163]:
# #write_ordered_dic('Datasets/LesMiserables.dot','ordered_LesMiserables.dot')
# show_graph('ordered_LesMiserables.dot','layered_ish',tree = 'BFS')

## DFS

In [164]:
# #write_ordered_dic('Datasets/LesMiserables.dot','ordered_LesMiserables.dot')
# show_graph('Datasets/LesMiserables.dot','layered_ish',tree = 'DFS')

By ordering the nodes based on their degree to select the best root node

In [165]:
#show_graph('ordered_LesMiserables.dot','layered_ish',tree = 'DFS')

## Radial

### Les Miserables

BFS Original network

In [166]:
#show_graph('Datasets/LesMiserables.dot','radial',tree = 'BFS')

BFS Network ordered so that the tree starts in the node with the highest degree

In [167]:
# #write_ordered_dic('Datasets/LesMiserables.dot','ordered_LesMiserables.dot')
# show_graph('ordered_LesMiserables.dot','radial',tree = 'BFS')

DFS Original network

In [168]:
# show_graph('Datasets/LesMiserables.dot','radial',tree = 'DFS')

### Jazz Network

BFS Original network

In [169]:
# show_graph('Datasets/JazzNetwork.dot','radial',tree = 'BFS')

BFS Network ordered so that the tree starts in the node with the highest degree

In [170]:
# write_ordered_dic('Datasets/JazzNetwork.dot','ordered_JazzNetwork.dot')
# show_graph('ordered_JazzNetwork.dot','radial',tree = 'BFS')

DFS Original network

In [171]:
#show_graph('Datasets/JazzNetwork.dot','radial',tree = 'DFS')